写在前面：  
  思路上采用STO-3G基组，氢原子用1s的Gaussian型轨道，碳原子用1s, 2s, 2px, 2py, 2pz对各单电子轨道进行表示，关于轨道的积分是先用smypy运算相应的符号函数，转化成数值函数后再进行全空间的数值积分。由于符号运算很慢，基本上可以说是跑不出来的。和同学讨论过，用球坐标应该会快一些，但是也没有本质解决问题，所以就懒得改了。  
  写这个程序前还不会python，新鲜现学的，恳请助教老师高抬贵手QAQ

In [14]:
import numpy as np
from scipy import integrate as intg
from scipy import linalg as li
import sympy as sp

In [15]:
h_alpha=np.array([0.3425250914E+01, 0.6239137298E+00, 0.1688554040E+00])
h_coeff=np.array([0.1543289673E+00,0.5353281423E+00,0.4446345422E+00])
c_alpha=np.array([[0.7161683735E+02,0.1304509632E+02,0.3530512160E+01],
                 [0.2941249355E+01,0.6834830964E+00,0.2222899159E+00],
                 [0.2941249355E+01,0.6834830964E+00,0.2222899159E+00]])#前两个是s轨道，后面是p轨道
c_coeff=np.array([[0.1543289673E+00,0.5353281423E+00,0.4446345422E+00],
                   [-0.9996722919E-01,0.3995128261E+00,0.7001154689E+00],
                 [0.1559162750E+00,0.6076837186E+00,0.3919573931E+00]])
x, y, z = sp.symbols('x y z')
x1,y1,z1= sp.symbols('x1 y1 z1')

$
\phi_s=c_1 (\frac{2\alpha_1}{\pi})^{0.75}e^{-\alpha_1 r^2}+c_2 (\frac{2\alpha_2}{\pi})^{0.75}e^{-\alpha_2 r^2}
+c_3 (\frac{2\alpha_3}{\pi})^{0.75}e^{-\alpha_3 r^2}
$

In [16]:
def func_s(a, b):
    return a[0]* (2*b[0]/np.pi)**0.75*sp.exp(-b[0]*(x**2+y**2+z**2))+\
    a[1]* (2*b[1]/np.pi)**0.75*sp.exp(-b[1]*(x**2+y**2+z**2))+\
    a[2]* (2*b[2]/np.pi)**0.75*sp.exp(-b[2]*(x**2+y**2+z**2))
    
g=func_s(h_coeff,h_alpha)
sp.pprint(g)
print(type(g))

                                  2                2                2                                      2           ↪
                   - 3.425250914⋅x  - 3.425250914⋅y  - 3.425250914⋅z                       - 0.6239137298⋅x  - 0.62391 ↪
0.276934355088743⋅ℯ                                                   + 0.267838851618852⋅ℯ                            ↪

↪        2                 2                                      2                2                2
↪ 37298⋅y  - 0.6239137298⋅z                        - 0.168855404⋅x  - 0.168855404⋅y  - 0.168855404⋅z 
↪                            + 0.0834736711327624⋅ℯ                                                  
<class 'sympy.core.add.Add'>


In [17]:
def func_px(a, b):
    f = a[0]* (2**7*b[0]**5/np.pi**3)**0.25*sp.exp(-b[0]*(x**2+y**2+z**2))*x
    return f

g=func_px(h_coeff,h_alpha)
sp.pprint(g)


def func_py(a, b):
    f = a[1]* (2**7*b[1]**5/np.pi**3)**0.25*sp.exp(-b[1]*(x**2+y**2+z**2))*y
    return f

def func_pz(a, b):
    f = a[2]* (2**7*b[2]**5/np.pi**3)**0.25*sp.exp(-b[2]*(x**2+y**2+z**2))*z
    return f

                                   2                2                2
                    - 3.425250914⋅x  - 3.425250914⋅y  - 3.425250914⋅z 
1.02506882711096⋅x⋅ℯ                                                  


\begin{align}
\phi_x=c_1 (\frac{2^7 {\alpha_1}^5}{\pi^3})^{0.25}e^{-\alpha_1 r^2} x\\
\phi_y=c_2 (\frac{2^7 {\alpha_2}^5}{\pi^3})^{0.25}e^{-\alpha_2 r^2} y\\
\phi_z=c_3 (\frac{2^7 {\alpha_3}^5}{\pi^3})^{0.25}e^{-\alpha_3 r^2} z
\end{align}

In [1]:
def veff(func1, func2):#可以跑但是巨慢无比，球坐标表达式会更快
    '''有效势积分，包含动能和核-电子能'''
    # 生成符号表达式
    intf = -0.5 * func1 * (
        sp.diff(func2, x, x) +
        sp.diff(func2, y, y) +
        sp.diff(func2, z, z)
    ) - func1 * (x**2+y**2+z*2)**(-0.5)*func2#有奇点
    
    # 将符号表达式转换为数值化的函数
    intf_numeric = sp.lambdify((x, y, z), intf, 'numpy')
    
    # 使用 tplquad 进行数值积分
    result, error = intg.tplquad(
        intf_numeric,  # 传递数值化函数
        -np.inf, np.inf,  # x 的积分范围
        lambda x: -np.inf, lambda x: np.inf,  # y 的积分范围
        lambda x, y: -np.inf, lambda x, y: np.inf  # z 的积分范围
    )
    
    return result

def veff_m(basis): #例如可以以c_basis为变量
    nbasis=len(basis)
    T=np.zeros([nbasis,nbasis])
    for i in range(nbasis):
        for j in range(nbasis):
            T[i,j]=veff(basis[i],basis[j])
    return T

\begin{align}
T[i,j]=\iiint \phi_i (-\frac{1}{2} \nabla^2 + \frac{1}{r}) \phi_j dx dydz
\end{align}  
其中$T[i,j]$为有效势矩阵T的ij元

In [19]:

c_basis=np.array([func_s(c_coeff[0],c_alpha[0]),
                  func_s(c_coeff[1],c_alpha[1]),
                  func_px(c_coeff[2],c_alpha[2]),
                  func_py(c_coeff[2],c_alpha[2]),
                  func_pz(c_coeff[2],c_alpha[2])],dtype=object)
#print(type(c_basis[0]))
#print(len(c_basis))

ca_c=np.array([[1.,0.,0.,0.],
            [0.,1.,0.,0.],
            [0.,0.,1.,0.],
            [0.,0.,0.,1.],
            [0.,0.,0.,0.]])#初猜系数矩阵
cb_c=np.array([[1.,0.],
            [0.,1.],
            [0.,0.],
            [0.,0.],
            [0.,0.]])
pa_c=np.dot(ca_c,ca_c.T)#初猜密度矩阵
pb_c=np.dot(cb_c,cb_c.T)
print(np.dot(c_basis,ca_c))


[2.70781441633193*exp(-71.61683735*x**2 - 71.61683735*y**2 - 71.61683735*z**2) + 2.61888021535572*exp(-13.04509632*x**2 - 13.04509632*y**2 - 13.04509632*z**2) + 0.816190573361246*exp(-3.53051216*x**2 - 3.53051216*y**2 - 3.53051216*z**2)
 -0.160017186731458*exp(-2.941249355*x**2 - 2.941249355*y**2 - 2.941249355*z**2) + 0.214035914467398*exp(-0.6834830964*x**2 - 0.6834830964*y**2 - 0.6834830964*z**2) + 0.161536097534647*exp(-0.2222899159*x**2 - 0.2222899159*y**2 - 0.2222899159*z**2)
 0.856044506407*x*exp(-2.941249355*x**2 - 2.941249355*y**2 - 2.941249355*z**2)
 0.538303757554759*y*exp(-0.6834830964*x**2 - 0.6834830964*y**2 - 0.6834830964*z**2)]


In [12]:
#测试 veff
result=veff(c_basis[0],c_basis[0])
print(result)

KeyboardInterrupt: 

In [97]:
#测试
'''
def func1_expr():
    return 0.1543289673 * sp.exp(-71.61683735 * (x**2 + y**2 + z**2))

def func2_expr():
    return 0.5353281423 * sp.exp(-13.04509632 * (x**2 + y**2 + z**2))

# Kinetic energy 表达式生成函数，接受符号表达式作为参数
def kinetic(func1, func2):
    # 直接使用传入的符号表达式
    kinetic_energy_expr = -0.5 * func1 * (
        sp.diff(func2, x, x) +
        sp.diff(func2, y, y) +
        sp.diff(func2, z, z)
    )
    
    # 将符号表达式转换为可数值化的函数
    kinetic_energy_numeric = sp.lambdify((x, y, z), kinetic_energy_expr, 'numpy')
    
    # 使用 tplquad 进行数值积分
    result, error = intg.tplquad(
        kinetic_energy_numeric,  # 传递可调用的数值化函数
        0.0, 100.0,  # x 的积分范围
        lambda x: 0.0, lambda x: 100.0,  # y 的积分范围
        lambda x, y: 0.0, lambda x, y: 100.0  # z 的积分范围
    )
    
    return result

# 获取 func1 和 func2 的符号表达式
func1 = func1_expr()
func2 = func2_expr()

# 调用 kinetic 函数
result = kinetic(func1, func2)
print("数值积分结果:", result)
结果能通过
'''

数值积分结果: 0.002443799571843324


In [1]:
def dintg(func1,func2,func3,func4):
    '''运算双电子积分<12|34>'''
    func2a = func2.subs({x: x1, y: y1, z: z1})
    func4a = func2.subs({x: x1, y: y1, z: z1})
    intf = func1 * func2a * func3 * func4a / ( (x-x1) ** 2 + (y-y1) ** 2 + (z-z1) ** 2)

    intf_numeric = sp.lambdify((x, y, z, x1, y1, z1), intf, 'numpy')

    # 设置积分范围，负无穷到正无穷
    x_bounds = [-np.inf, np.inf]
    y_bounds = [-np.inf, np.inf]
    z_bounds = [-np.inf, np.inf] 
    x1_bounds = [-np.inf, np.inf]
    y1_bounds = [-np.inf, np.inf]
    z1_bounds = [-np.inf, np.inf]

    result, error = intg.nquad(
        intf_numeric,
        [x_bounds,y_bounds,z_bounds,x1_bounds,y1_bounds,z1_bounds])

    return result

def coulomb_m(pa): #or pb
    '''运算库伦矩阵'''
    nbasis=pa.shape[0]
    J=np.zeros([nbasis,nbasis])
    for i in range(nbasis):
        for j in range(nbasis):
            for u in range(nbasis):
                for v in range(nbasis):
                    J[i, j] += (pa[u, v])*dintg(basis[i],basis[v],basis[j],basis[u])
    return J

def exchange_ma(pa): 
    '''运算alpha电子的交换矩阵Ka'''
    nbasis=pa.shape[0]
    Ka=np.zeros([nbasis,nbasis])
    for i in range(nbasis):
        for j in range(nbasis):
            for u in range(nbasis):
                for v in range(nbasis):
                    Ka[i, j] += pa[u, v]*dintg(basis[i],basis[v],basis[u],basis[j])
    return Ka

def exchange_mb(pb): 
    '''运算beta电子的交换矩阵Kb'''
    nbasis=pb.shape[0]
    Kb=np.zeros([nbasis,nbasis])
    for i in range(nbasis):
        for j in range(nbasis):
            for u in range(nbasis):
                for v in range(nbasis):
                    Kb[i, j] += pb[u, v]*dintg(basis[i],basis[v],basis[u],basis[j])
    return Kb

定义基本双电子积分<ij|mn>:  
\begin{align}
<ij|mn>=\int \phi_i^*(r_1)\phi_i^*(r_2)\frac{1}{r_{12}}\phi_m(r_1)\phi_n(r_2) dr_1dr_2
\end{align}  
对于库伦矩阵J，ij元为:  
\begin{align}
J[i, j]=\sum_{m,n} P[n,m]<im|jn> 
\end{align}  
对于交换矩阵K，ij元为:  
\begin{align}
K[i, j]=\sum_{m,n} P[n,m]<im|nj> 
\end{align}  
其中P为密度矩阵，在前面出现过了

In [2]:
def overlap(func1,func2):
    intf = func1*func2
    intf_numerical= sp.lambdify((x,y,z),intf,'numpy')
    result, error = intg.tplquad(
        intf_numeric,  # 传递数值化函数
        -np.inf, np.inf,  # x 的积分范围
        lambda x: -np.inf, lambda x: np.inf,  # y 的积分范围
        lambda x, y: -np.inf, lambda x, y: np.inf  # z 的积分范围
    )
    return result
    
def overlap_m(basis):
    '''计算重叠矩阵'''
    nbasis = len(molecule)
    S = np.zeros([nbasis, nbasis])
    for i in range(nbasis):
        for j in range(nbasis):
            S[i,j]=overlap(basis[i],basis[j])
    return S

对于重叠矩阵S, ij元为：  
\begin{align}
S[i,j]=\int \phi_i \phi_j dr
\end{align}  


In [3]:
def comp_p(mo,c):#计算密度矩阵,c是初始系数矩阵
    nbasis_functions = c.shape[0]
    p_m = np.zeros((nbasis_functions, nbasis_functions))
    for i in range(nbasis_functions):
        for j in range(nbasis_functions):
            # mo is (natomic_orbtials x nMOs)
            for l in range(c.shape[1]):
            # for l in range(number_occupied_orbitals):
                C = mo[i,l]
                C_dagger = mo[j,l]
                density_matrix[i,j] += occupation * C * C_dagger
    return density_matrix

本篇初始系数矩阵记为c,其行数（c.shape[0]）等于基函数数量 (nbasis)，列数(c.shape[1])是占据轨道的数量。中间SCF循环中系数矩阵记为mo，mo为nbasis×nbasis的矩阵，但事实上算密度矩阵的时候只需要用到前几个占据轨道，所以写这个计算密度矩阵的模块相当于是把mo矩阵的后几列舍弃，变成nbasis×c.shape[1]的矩阵后计算密度矩阵。

In [5]:
def comp_e(moa, mob, T, Ka, Kb, ca, cb, basis):
    pa=comp_p(moa,ca)
    pb=comp_p(mob,cb)
    elec_energy=0.
    nbasis=ca.shape[0]
    for i in range(nbasis):
        for j in range(nbasis):
            elec_energy += (pa[i,j]+pb[i,j])*T[i,j]+0.5*coulomb_m(pa)[i,j]*pa[i,j]+\
            0.5*coulomb_m(pb)[i,j]*pb[i,j]-0.5*Ka[i,j]*pa[i,j]-0.5*Kb[i,j]*pb[i,j]

    for u in range(nbasis):
        for v in range(nbasis):
            for p in range(nbasis):
                for q in range(nbasis):
                    for m in range(ca.shape[1]):
                        for n in range(cb.shape[1]):
                            elec_energy += moa[u,m]*moa[p,m]*mob[v,n]*mob[q,n]*dintg(basis[u],basis[v],basis[p],basis[q])

    return elec_energy

计算电子能量$E_0$:  
\begin{align}
E_0=\sum_i <i|h|i> + \frac{1}{2}\sum_{m,n} <mn|mn>-<mn|nm>
\end{align}  
须知：这里的i,m,n为单电子本征态$\psi_i, \psi_m, \psi_n$而非基函数$\phi$。但在实际计算的时候需要展开成基函数表象。考虑到电子自旋，<mn|mn>的求和中，m/n可以为同种自旋，也可以为异种自旋。<mn|nm>的求和中，m/n必须为同种自旋。核函数部分能量为：  
\begin{align}
\sum_{i,j} (P_{\alpha}[i,j]+P_{\beta}[i,j])T[i,j]
\end{align}  
同自旋库伦项的求和为：  
\begin{align}
\sum_{i,j} J_{\sigma}[i,j]P_{\sigma}[i,j]
\end{align}
同自旋交换项的求和为：  
\begin{align}
\sum_{i,j} K_{\sigma}[i,j]P_{\sigma}[i,j]
\end{align}  
不同自旋库伦项也不为0。这部分由于没有现成的模块化矩阵可以用所以干脆自己重新写了一个六重求和（见上）

In [7]:
def scf_hf(matrix, parameters, basis):
    S, T, pa, pb, ca, cb = matrix
    tolerance, maxsteps = parameters
    elec_energy = 0.
    nbasis=len(basis)
    na=ca.shape[1]+1#占据轨道和第一个非占据
    nb=cb.shape[1]+1

    for scf_step in range (maxsteps):
        elec_energy_old = elec_energy
        Ka=exchange_ma(pa)
        Kb=exchange_mb(pb)
        J=coulomb_m(pa)+coulomb_m(pb)
        #计算初始Fock矩阵
        Fa= T+J+Ka
        Fb= T+J+Kb

        S_inv=li.inv(S)
        S_inv_sqrt= li.sqrtm(S_inv)
        #计算正交归一化基组对应的Fock矩阵
        Fa_us=np.dot(S_inv_sqrt,np.dot(Fa,S_inv_sqrt))
        eigenvalues_a, eigenvectors_a = li.eigh(Fa_us)
        Fb_us=np.dot(S_inv_sqrt,np.dot(Fb,S_inv_sqrt))
        eigenvalues_b, eigenvectors_b = li.eigh(Fb_us)
        moa=np.dot(S_inv_sqrt,eigenvectors_a)
        mob=np.dot(S_inv_sqrt,eigenvectors_b)

        pa=comp_p(moa,ca)
        pb=comp_p(mob,cb)

        elec_energy=comp_e(moa, mob, T, Ka, Kb, ca, cb, basis)

        top_na_e=eigenvalues_a[:na]
        top_nb_e=eigenvalues_b[:nb]
        top_na_o=eigenvectors_a[:,:na]
        top_nb_o=eigenvectors_b[:,:nb]

        if abs(electronic_energy-electronic_energy_old) < tolerance:
            return top_na_e, top_nb_e, top_na_o, top_nb_o

        print("未在最大次数内收敛")
        return top_na_e, top_nb_e, top_na_o, top_nb_o

参考蒋鸿《现代量子化学导论》：  
1. 确定分子几何结构和基组。  
2. 计算所有单电子和双电子积分，K, S等 。  
3. 对角化重叠矩阵S，确定正交归一化矩阵X。  
4. 产生密度矩阵P的初始猜测。  
5. 由密度矩阵P 计算Fock矩阵P 。  
6. 计算正交归一化基组对应的Fock矩阵F'  
7. 对角化F'，得到C'和{εi}。  
8. 计算轨道展开系数C = XC'  
9. 由C计算新的密度矩阵P  
10. 判断是否收敛,如未收敛，回到第5步。

In [9]:
def exc_lda(p,basis):#已经*0.5了
    n0=0
    nbasis = len(basis)
    K_lda=np.zeros([nbasis,nbasis])
    for i in range(nbasis):
        for j in range(nbasis):
            n0 += p[m,n]*basis[m]*basis[n]

    vxc= -(6*np.pi*n0)**(1/3)/np.pi

    for m in range(nbasis):
        for n in range(nbasis):
            vxc_numeric=sp.lambdify((x,y,z),vxc,'numpy')
            result, error = intg.tplquad(
            vxc_numeric,  
            -np.inf, np.inf,  
            lambda x: -np.inf, lambda x: np.inf,  
            lambda x, y: -np.inf, lambda x, y: np.inf)
            K_lda[m,n]=result

    return K_lda

def comp_e_lda(pa, pb, T, Ka, Kb, ca, cb, basis):
    elec_energy=0.
    nbasis=ca.shape[0]
    for i in range(nbasis):
        for j in range(nbasis):
            elec_energy += (pa[i,j]+pb[i,j])*T[i,j]+coulomb_m(pa)[i,j]*pa[i,j]+\
            0.5*coulomb_m(pb)[i,j]*pb[i,j]-Ka[i,j]*pa[i,j]-Kb[i,j]*pb[i,j]

    return elec_energy
    

由spin scaling relation有：  
\begin{align}
E_x [n_{\alpha},n_{\beta}]=\frac{1}{2} (E_x [2n_{\alpha}],E_x [2n_{\beta}])
\end{align}  
对于LDA,忽略correlation部分:
\begin{align}
V_{xc}^{\sigma}=\frac{\delta E_{xc}[n,\nabla n]}{\delta n_{\sigma}(r)}=\frac{\delta E_{xc}[2n_\sigma]}{\delta 2n_{\sigma}(r)}
\end{align}
LDA有形式：
\begin{align}
E_x^{LDA}[n]=-\frac{3(3\pi)^{1/3}}{4\pi}\int d^3rn(r)^{4/3}
\end{align}
所以
\begin{align}
V_{xc}^{\sigma}=-\frac{(6\pi n_\sigma)^{\frac{1}{3}}}{\pi}
\end{align}

In [11]:
def scf_lda(matrix, parameters, basis):
    S, T, pa, pb, ca, cb = matrix
    tolerance, maxsteps = parameters
    elec_energy = 0.
    nbasis=len(basis)
    na=ca.shape[1]+1#占据轨道和第一个非占据
    nb=cb.shape[1]+1

    for scf_step in range (maxsteps):
        elec_energy_old = elec_energy
        Ka=exc_lda(pa,basis)
        Kb=exc_lda(pb,basis)
        J=coulomb_m(pa)+coulomb_m(pb)
        #计算初始Fock矩阵
        Fa= T+Ka+J
        Fb= T+Kb+J

        S_inv=li.inv(S)
        S_inv_sqrt= li.sqrtm(S_inv)
        #计算正交归一化基组对应的Fock矩阵
        Fa_us=np.dot(S_inv_sqrt,np.dot(Fa,S_inv_sqrt))
        eigenvalues_a, eigenvectors_a = li.eigh(Fa_us)
        Fb_us=np.dot(S_inv_sqrt,np.dot(Fb,S_inv_sqrt))
        eigenvalues_b, eigenvectors_b = li.eigh(Fb_us)
        moa=np.dot(S_inv_sqrt,eigenvectors_a)
        mob=np.dot(S_inv_sqrt,eigenvectors_b)

        pa=comp_p(moa,ca)
        pb=comp_p(mob,cb)

        elec_energy=comp_e_lda(moa, mob, T, Ka, Kb, ca, cb, basis)

        top_na_e=eigenvalues_a[:na]
        top_nb_e=eigenvalues_b[:nb]
        top_na_o=eigenvectors_a[:,:na]
        top_nb_o=eigenvectors_b[:,:nb]

        if abs(electronic_energy-electronic_energy_old) < tolerance:
            return top_na_e, top_nb_e, top_na_o, top_nb_o

        print("未在最大次数内收敛")
        return top_na_e, top_nb_e, top_na_o, top_nb_o

In [ ]:
S_c=overlap(c_basis)
T_c=veff_m(c_basis)
parameters=[1e-5,15]
matrix_c=[S_c, T_c, pa_c, pb_c, ca_c, cb_c]
c_ea_hf, c_eb_hf, c_oa_hf, c_ob_hf=scf_hf(matrix_c,parameters,c_basis)
print(f'hf碳原子α自旋电子的占据能级和LUMO能级为{c_ea_hf}')
print(f'hf碳原子β自旋电子的占据能级和LUMO能级为{c_eb_hf}')
print(f'hf碳原子α自旋电子的占据轨道和LUMO轨道为（列向量为基函数系数）{c_oa_hf}')
print(f'hf碳原子β自旋电子的占据轨道和LUMO轨道为（列向量为基函数系数）{c_ob_hf}')
c_ea_lda, c_eb_lda, c_oa_lda, c_ob_lda=scf_lda(matrix_c,parameters,c_basis)
print(f'lda碳原子α自旋电子的占据能级和LUMO能级为{c_ea_lda}')
print(f'lda碳原子β自旋电子的占据能级和LUMO能级为{c_eb_lda}')
print(f'lda碳原子α自旋电子的占据轨道和LUMO轨道为（列向量为基函数系数）{c_oa_lda}')
print(f'lda碳原子β自旋电子的占据轨道和LUMO轨道为（列向量为基函数系数）{c_ob_lda}')

h_basis=np.array([func_s(h_coeff[0],h_alpha[0])],dtype=object)
ca_h=np.array([1.])
cb_h=np.array([0.])
pa_h=np.dot(ca_h,ca_h.T)
pb_h=np.dot(cb_h,cb_h.T)
S_h=overlap(h_basis)
T_c=veff_m(h_basis)
matrix_h=[S_h,T_h,pa_h,pb_h,ca_h,cb_h]
h_ea_hf, h_eb_hf, h_oa_hf, h_ob_hf=scf_hf(matrix_h,parameters,h_basis)
print(f'氢原子α自旋电子的占据能级和LUMO能级为{h_ea_hf}')
print(f'氢原子β自旋电子的占据能级和LUMO能级为{h_eb_hf}')
print(f'氢原子α自旋电子的占据轨道和LUMO轨道为（列向量为基函数系数）{h_oa_hf}')
print(f'氢原子β自旋电子的占据轨道和LUMO轨道为（列向量为基函数系数）{h_ob_hf}')
h_ea_lda, h_eb_lda, h_oa_lda, h_ob_lda=scf_lda(matrix_h,parameters,h_basis)
print(f'lda氢原子α自旋电子的占据能级和LUMO能级为{h_ea_lda}')
print(f'lda氢原子β自旋电子的占据能级和LUMO能级为{h_eb_lda}')
print(f'lda氢原子α自旋电子的占据轨道和LUMO轨道为（列向量为基函数系数）{h_oa_lda}')
print(f'lda氢原子β自旋电子的占据轨道和LUMO轨道为（列向量为基函数系数）{h_ob_lda}')

参考资料：https://github.com/yangdatou/hf-tutorial, https://github.com/theochem/tinydft,tinydft应该是打格点，实际上用的不多。  
https://bohrium.dp.tech/notebooks/2022204647，这个真的很有用。  
(Attila Szabo, Neil S. Ostlund)Modern Quantum Chemistry Introduction to Advanced Electronic Structure Theory, 现代量子化学导论 (蒋鸿), 量子化学（徐光宪）。第一本是我最开始学习用书，第二本写的很有逻辑，而且是中文方便参考。徐光宪量子化学东西真的很全，可以当字典用。网上翻了复数本书，最后回过头发现徐光宪上就有。 
感谢gpt提供正确提示的同时混入一些错误。只有有错误你才知道你用的是人工智能（bushi  
感谢zjp同学的讨论与帮助，如果没有和他一起选电子结构理论与计算这门课我可能心态早就崩了（虽然现在也崩了）。感谢xzn学长的倾情帮助，如果没有他的帮助这个程序可能连头都没有开，以及很惭愧没有用他努力写出来的提前把积分算出解析式的真正能跑出来的程序。感谢我的对象听我说垃圾话，包容我的负面情绪（x  
可能的改进是提前把积分算出解析式，或者用格点。这个程序只是具有形式，跑不出来。